In [5]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

In [10]:
tips = sns.load_dataset('tips')
print(tips)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


In [18]:
model = smf.ols(formula='tip~total_bill',data=tips)

In [20]:
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    tip   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     203.4
Date:                Wed, 02 Apr 2025   Prob (F-statistic):           6.69e-34
Time:                        18:44:09   Log-Likelihood:                -350.54
No. Observations:                 244   AIC:                             705.1
Df Residuals:                     242   BIC:                             712.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9203      0.160      5.761      0.0

In [23]:
print(results.params)

Intercept     0.920270
total_bill    0.105025
dtype: float64


In [26]:
print(results.conf_int())

                   0         1
Intercept   0.605622  1.234918
total_bill  0.090517  0.119532


In [ ]:
************************************

In [29]:
titanic=pd.read_csv('titanic.csv')
print(titanic)

     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alo

In [32]:
titanic_sub=(
    titanic[["survived","sex","age","embarked"]].copy().dropna()
)
print(titanic_sub)

     survived     sex   age embarked
0           0    male  22.0        S
1           1  female  38.0        C
2           1  female  26.0        S
3           1  female  35.0        S
4           0    male  35.0        S
..        ...     ...   ...      ...
885         0  female  39.0        Q
886         0    male  27.0        S
887         1  female  19.0        S
889         1    male  26.0        C
890         0    male  32.0        Q

[712 rows x 4 columns]


In [35]:
print(titanic_sub["survived"].value_counts())

survived
0    424
1    288
Name: count, dtype: int64


In [38]:
form ='survived~sex+age+embarked'
#fitting the logistic regression model

py_logistic_smf=smf.logit(formula =form, data=titanic_sub).fit()
print(py_logistic_smf.summary())


Optimization terminated successfully.
         Current function value: 0.509889
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  712
Model:                          Logit   Df Residuals:                      707
Method:                           MLE   Df Model:                            4
Date:                Wed, 02 Apr 2025   Pseudo R-squ.:                  0.2444
Time:                        20:07:25   Log-Likelihood:                -363.04
converged:                       True   LL-Null:                       -480.45
Covariance Type:            nonrobust   LLR p-value:                 1.209e-49
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         2.2046      0.322      6.851      0.000       1.574       2.835
sex[T.male]      -2.

In [43]:
import numpy as np
res_sm=pd.DataFrame(py_logistic_smf.params,columns=["coefs_sm"])

#calculate the odds
res_sm["odds_sm"]=np.exp(res_sm["coefs_sm"])

#round the decimals
print(res_sm.round(3))

               coefs_sm  odds_sm
Intercept         2.205    9.066
sex[T.male]      -2.476    0.084
embarked[T.Q]    -1.816    0.163
embarked[T.S]    -1.007    0.365
age              -0.008    0.992


In [46]:
titanic_dummy= pd.get_dummies(
    titanic_sub[["survived","sex","age","embarked"]],
    drop_first=True
)
print(titanic_dummy)

     survived   age  sex_male  embarked_Q  embarked_S
0           0  22.0      True       False        True
1           1  38.0     False       False       False
2           1  26.0     False       False        True
3           1  35.0     False       False        True
4           0  35.0      True       False        True
..        ...   ...       ...         ...         ...
885         0  39.0     False        True       False
886         0  27.0      True       False        True
887         1  19.0     False       False        True
889         1  26.0      True       False       False
890         0  32.0      True        True       False

[712 rows x 5 columns]


In [52]:
from sklearn import linear_model

py_logistic_sklearn1=(
    linear_model.LogisticRegression().fit(
        X=titanic_dummy.iloc[:,1:], #all columns expect 1st
        y=titanic_dummy.iloc[:,0] #just the 1st column
    )
)

#get the names of dummy variable 
dummy_names= titanic_dummy.columns.to_list()
sk1_res1=pd.DataFrame(
    py_logistic_sklearn1.intercept_,
    index=["Intercept"],
    columns=["coef_sk1"],
)
sk1_res2=pd.DataFrame(
    py_logistic_sklearn1.coef_.T,
    index=dummy_names[1:],
    columns=["coef_sk1"],
)

#put the results into a 1 DF to show the results

res_sklearn_pd_1=pd.concat([sk1_res1,sk1_res2])
#calculate the odds
res_sklearn_pd_1["odds_sk1"]=np.exp(res_sklearn_pd_1["coef_sk1"])
print(res_sklearn_pd_1.round(3))

            coef_sk1  odds_sk1
Intercept      2.024     7.568
age           -0.008     0.992
sex_male      -2.371     0.093
embarked_Q    -1.368     0.255
embarked_S    -0.887     0.412
